In [1]:
%load_ext autoreload
%autoreload 2

In [36]:
import os
from target_encoding import TargetEncoderClassifier, TargetEncoder
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from functools import reduce
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import cross_val_score
import json
from functools import partial
from dsb2019.models.coeff import ThresholdClassifier

from dsb2019.models.tracking import track_experiment, track_submission_info
from dsb2019.data.validation import InstallationFold, cross_validate, quad_kappa
from dsb2019.visualization import session_browser
from dsb2019.data import DATA_DIR
from dsb2019.models import MODELS_DIR
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm
import hyperopt
from hyperopt import hp, fmin, Trials, tpe, STATUS_OK
tqdm.pandas()
pd.options.display.max_rows=999

In [3]:
train = pd.read_csv(DATA_DIR / 'raw/train.csv')
test = pd.read_csv(DATA_DIR / 'raw/test.csv')
train_labels = pd.read_csv(DATA_DIR / 'raw/train_labels.csv')
submission = pd.read_csv(DATA_DIR / 'raw/sample_submission.csv')

In [4]:
games = ['Scrub-A-Dub', 'All Star Sorting', 'Mushroom Sorter (Assessment)',
       'Air Show', 'Crystals Rule', 'Bird Measurer (Assessment)',
       'Dino Drink', 'Bubble Bath', 'Dino Dive', 'Chow Time',
       'Cauldron Filler (Assessment)', 'Pan Balance', 'Happy Camel',
       'Cart Balancer (Assessment)', 'Chest Sorter (Assessment)',
       'Leaf Leader']


def unwrap_event_data(df):
    unwrapped=pd.DataFrame(data=list(df.event_data.apply(json.loads).values))
    return pd.concat([unwrapped.reset_index(),df.reset_index()],axis=1)


def process_installations(train_labels, train, process_log):
    result = []
    train=train.sort_values("timestamp")
    installations = train.groupby("installation_id")
    for i, game_session, title, installation_id, accuracy_group in tqdm(train_labels[["game_session", "title", "installation_id", "accuracy_group"]].itertuples(), 
                                                              total=len(train_labels)):
        player_log = installations.get_group(installation_id).reset_index()
        log_length = player_log[(player_log.game_session==game_session) & (player_log.title==title)].index[0]
        player_log = player_log.iloc[:(log_length + 1)]
        player_log["accuracy_group"] = accuracy_group
        player_log["target_game_session"] = game_session
        features = process_log(player_log)
        features["installation_id"] = installation_id
        features["accuracy_group"] = accuracy_group
        result.append(features)
    return pd.DataFrame(data=result)
 
    
def process_log(df):
    assessment_title=df.title.iloc[-1]    

    history = df.iloc[:-1]
    history = history[history.type.isin(["Game", "Assessment"])].copy()

    def calculate_ratios(df):
        n_correct=df.correct_move.sum()
        n_incorrect=df.wrong_move.sum()
        ratio=n_correct/(n_correct+n_incorrect)
        return n_correct, n_incorrect, ratio
    
    def make_move_stats(df, title,n_lags=2):
        df=df.copy()
        if len(df):
            df = unwrap_event_data(df)
        if "correct" in df.columns:
            df["correct_move"] = df.correct == True
            df["wrong_move"] = df.correct == False
        else:
            df["correct_move"]=False
            df["wrong_move"]=False
        result = []
        result.extend(zip([f"n_correct {title}", f"n_incorrect {title}", f"global_ratio {title}"], calculate_ratios(df)))
        if n_lags:
            last_sessions = df.game_session.unique()[-n_lags:]
            for i in range(n_lags):
                if i < len(last_sessions): 
                    result.extend(zip([f"n_correct {title} {i}", f"n_incorrect {title} {i}",f"ratio {title} {i}"], 
                                      calculate_ratios(df[df.game_session==last_sessions[i]])))
                else:
                    result.extend(zip([f"n_correct {title} {i}", f"n_incorrect {title} {i}",f"ratio {title} {i}"], [None, None, None]))
        return {k: v for k, v in result}
    result = {"title": games.index(assessment_title)}
    for game in games:
        stats=history[history.title==game]
        stats=make_move_stats(stats, game)
        result.update(stats)
    return result

In [5]:
train_features = process_installations(train_labels, train, process_log)

100%|██████████| 17690/17690 [44:36<00:00,  6.61it/s] 


In [6]:
train_features=train_features.fillna(-1)

In [41]:
def lgb_quad_kappa(preds, true):
    true = true.get_label()
    #preds = preds.reshape((4, -1)).argmax(axis=0)
    preds = np.rint(preds)
    preds = np.maximum(0, preds)
    preds = np.minimum(3, preds)
    return "quad_kappa", quad_kappa(true, preds), True
    
    
def train_baseline(x_train,y_train, params=None):
    x_train_all, x_val_all,y_train_all,y_val_all = train_test_split(
        x_train,y_train,
        test_size=0.15,
        random_state=2019,
    )
    train_set = lgb.Dataset(x_train_all, y_train_all)
    val_set = lgb.Dataset(x_val_all, y_val_all)

#     params = {
#         'learning_rate': 0.01,
#         'bagging_fraction': 0.9,
#         'feature_fraction': 0.9,
#         'num_leaves': 14,
#         'lambda_l1': 0.1,
#         'lambda_l2': 1,
#         'metric': 'multiclass',
#         'objective': 'multiclass',
#         'num_classes': 4,
#         'random_state': 2019
#     }

    return lgb.train(params, train_set, num_boost_round=10000, early_stopping_rounds=2000, valid_sets=[val_set], verbose_eval=100,
                    feval=lgb_quad_kappa)

def make_features(df):
    return df.drop(["installation_id", "accuracy_group"], axis=1), df.accuracy_group.values

def make_features_wrapper(train, test):
    def make_features(df):
        return df.drop(["installation_id", "accuracy_group"], axis=1), df.accuracy_group.values
    
    return make_features(train), make_features(test) 


def make_predictions(model,x_test_all,y_test):
    preds=model.predict(x_test_all)
    preds = np.rint(preds)
    preds = np.maximum(0, preds)
    preds = np.minimum(3, preds)
    return preds,y_test

In [8]:
def process_test_installations(test):
    test = test.sort_values("timestamp")
    test=test.groupby("installation_id").progress_apply(process_log).reset_index()
    test.columns = ["installation_id", "features"]
    result = []
    for i, installation_id, feature in test.itertuples():
        result.append(feature)
        feature["installation_id"]=installation_id
    return pd.DataFrame(result).fillna(-1)
test_features=process_test_installations(test)

100%|██████████| 1000/1000 [01:35<00:00, 10.47it/s]


In [24]:
subtrain_installations=pd.Series(train_features.installation_id.unique()).sample(frac=1., random_state=2019)
subtrain_features=train_features[train_features.installation_id.isin(subtrain_installations.values)].copy()
def check_hyperparams(params):
    print(params)
    if "max_depth" in params:
        params["max_depth"] = int(params["max_depth"])
    if "num_leaves" in params:
        params["num_leaves"] = int(params["num_leaves"])

    train_baseline_with_params = partial(train_baseline, params=params)
    cv=InstallationFold(n_splits=3)
    predictions = cross_validate(subtrain_features, subtrain_features.accuracy_group, make_features_wrapper, train_baseline_with_params, make_predictions,
                                cv=cv)
    return {
        "loss": -np.mean([quad_kappa(true, pred) for pred, true in predictions]),
        "status": STATUS_OK,
        "params": params
    }


def tune(check_params, n_tries=25, n_learning_rate_tries=15, learning_rate=None, n_estimators=10_000):        
    if learning_rate is None:
        learning_rate_space = {
            'learning_rate': hp.loguniform("learning_rate", np.log(0.005), np.log(0.3)),
            'metric': 'rmse',
            'objective': 'rmse',
            #'num_classes': 4,
            'random_state': 2019,
            "n_estimators": n_estimators,

        }
        trials = Trials()
        result = fmin(check_params,
                      learning_rate_space, tpe.suggest, n_learning_rate_tries, trials=trials)
        print(result)
        learning_rate = round(trials.best_trial["result"]["params"]["learning_rate"], 3)

    param_space = {
        'metric': 'rmse',
        'objective': 'rmse',
        #'num_classes': 4,
        'lambda_l1': hp.uniform("lamba_l1", 1e-10, 1),
        'lambda_l2': hp.uniform("lambda_l2", 1e-10, 1),
        'random_state': 2019,
        "n_estimators": n_estimators,
        "learning_rate": learning_rate,
        "max_depth": hp.quniform("max_depth", 2, 16, 1),
        "num_leaves": hp.choice("num_leaves", [3, 7, 15, 31, 63, 127, 255, 511, 1023, 2047, 4095]),
        "subsample": hp.quniform("subsample", 0.01, 1, 0.01),
        "feature_fraction": hp.quniform("feature_fraction", 0.01, 1, 0.01),
    }

    trials = Trials()
    fmin(check_params,
         param_space, tpe.suggest, n_tries, trials=trials)
    best_params = trials.best_trial["result"]["params"]
    return best_params

In [25]:
best_params=tune(check_hyperparams, n_tries=25, n_learning_rate_tries=10)

{'learning_rate': 0.019989094607947705, 'metric': 'rmse', 'n_estimators': 10000, 'objective': 'rmse', 'random_state': 2019}
Training until validation scores don't improve for 2000 rounds
  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[100]	valid_0's rmse: 1.02289	valid_0's quad_kappa: 0.384783
[200]	valid_0's rmse: 1.00579	valid_0's quad_kappa: 0.494125
[300]	valid_0's rmse: 1.00266	valid_0's quad_kappa: 0.505066
[400]	valid_0's rmse: 1.00316	valid_0's quad_kappa: 0.513203
[500]	valid_0's rmse: 1.00457	valid_0's quad_kappa: 0.510493
[600]	valid_0's rmse: 1.00644	valid_0's quad_kappa: 0.509142
[700]	valid_0's rmse: 1.00821	valid_0's quad_kappa: 0.513988
[800]	valid_0's rmse: 1.00926	valid_0's quad_kappa: 0.509968
[900]	valid_0's rmse: 1.01121	valid_0's quad_kappa: 0.507195
[1000]	valid_0's rmse: 1.01358	valid_0's quad_kappa: 0.508987
[1100]	valid_0's rmse: 1.01531	valid_0's quad_kappa: 0.508587
[1200]	valid_0's rmse: 1.01636	valid_0's quad_kappa: 0.507422
[1300]	valid_0's rmse: 1.01851	valid_0's quad_kappa: 0.503885
[1400]	valid_0's rmse: 1.0205	valid_0's quad_kappa: 0.503093
[1500]	valid_0's rmse: 1.02233	valid_0's quad_kappa: 0.505719
[1600]	valid_0's rmse: 1.02422	valid_0's quad_kappa: 0.511575
[1700]	valid_0's r

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[100]	valid_0's rmse: 1.02238	valid_0's quad_kappa: 0.387174                  
[200]	valid_0's rmse: 1.00644	valid_0's quad_kappa: 0.495172                  
[300]	valid_0's rmse: 1.00295	valid_0's quad_kappa: 0.50423                   
[400]	valid_0's rmse: 1.00331	valid_0's quad_kappa: 0.505404                  
[500]	valid_0's rmse: 1.00404	valid_0's quad_kappa: 0.513888                  
[600]	valid_0's rmse: 1.00572	valid_0's quad_kappa: 0.517709                  
[700]	valid_0's rmse: 1.0077	valid_0's quad_kappa: 0.514686                   
[800]	valid_0's rmse: 1.00905	valid_0's quad_kappa: 0.507445                  
[900]	valid_0's rmse: 1.01107	valid_0's quad_kappa: 0.505498                  
[1000]	valid_0's rmse: 1.01261	valid_0's quad_kappa: 0.506178                 
[1100]	valid_0's rmse: 1.01528	valid_0's quad_kappa: 0.51276                  
[1200]	valid_0's rmse: 1.01822	valid_0's quad_kappa: 0.508923                 
[1300]	valid_0's rmse: 1.02048	valid_0's quad_kappa:

...

[100]	valid_0's rmse: 1.00436	valid_0's quad_kappa: 0.510126                     
[200]	valid_0's rmse: 1.00878	valid_0's quad_kappa: 0.51367                      
[300]	valid_0's rmse: 1.01844	valid_0's quad_kappa: 0.519868                     
[400]	valid_0's rmse: 1.02438	valid_0's quad_kappa: 0.520855                     
[500]	valid_0's rmse: 1.02726	valid_0's quad_kappa: 0.517866                     
[600]	valid_0's rmse: 1.03176	valid_0's quad_kappa: 0.514298                     
[700]	valid_0's rmse: 1.03455	valid_0's quad_kappa: 0.51025                      
[800]	valid_0's rmse: 1.03765	valid_0's quad_kappa: 0.509823                     
[900]	valid_0's rmse: 1.0408	valid_0's quad_kappa: 0.51108                       
[1000]	valid_0's rmse: 1.0439	valid_0's quad_kappa: 0.509499                     
[1100]	valid_0's rmse: 1.04597	valid_0's quad_kappa: 0.508429                    
[1200]	valid_0's rmse: 1.04849	valid_0's quad_kappa: 0.506021                    
[1300]	valid_0's

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[100]	valid_0's rmse: 1.00787	valid_0's quad_kappa: 0.480957                     
[200]	valid_0's rmse: 1.00927	valid_0's quad_kappa: 0.508331                     
[300]	valid_0's rmse: 1.0171	valid_0's quad_kappa: 0.500257                      
[400]	valid_0's rmse: 1.0228	valid_0's quad_kappa: 0.500446                      
[500]	valid_0's rmse: 1.02665	valid_0's quad_kappa: 0.498726                     
[600]	valid_0's rmse: 1.02947	valid_0's quad_kappa: 0.500018                     
[700]	valid_0's rmse: 1.03357	valid_0's quad_kappa: 0.499528                     
[800]	valid_0's rmse: 1.03707	valid_0's quad_kappa: 0.498381                     
[900]	valid_0's rmse: 1.03891	valid_0's quad_kappa: 0.492301                     
[1000]	valid_0's rmse: 1.04109	valid_0's quad_kappa: 0.495866                    
[1100]	valid_0's rmse: 1.04351	valid_0's quad_kappa: 0.494099                    
[1200]	valid_0's rmse: 1.04567	valid_0's quad_kappa: 0.496791                    
[1300]	valid_0's

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[100]	valid_0's rmse: 1.00486	valid_0's quad_kappa: 0.511099                     
[200]	valid_0's rmse: 1.0097	valid_0's quad_kappa: 0.513506                      
[300]	valid_0's rmse: 1.01796	valid_0's quad_kappa: 0.511175                     
[400]	valid_0's rmse: 1.02352	valid_0's quad_kappa: 0.511055                     
[500]	valid_0's rmse: 1.02844	valid_0's quad_kappa: 0.507082                     
[600]	valid_0's rmse: 1.03208	valid_0's quad_kappa: 0.50714                      
[700]	valid_0's rmse: 1.03515	valid_0's quad_kappa: 0.509761                     
[800]	valid_0's rmse: 1.03755	valid_0's quad_kappa: 0.509766                     
[900]	valid_0's rmse: 1.03997	valid_0's quad_kappa: 0.506428                     
[1000]	valid_0's rmse: 1.04305	valid_0's quad_kappa: 0.508735                    
[1100]	valid_0's rmse: 1.04549	valid_0's quad_kappa: 0.506196                    
[1200]	valid_0's rmse: 1.04777	valid_0's quad_kappa: 0.507532                    
[1300]	valid_0's

In [26]:
best_params

{'feature_fraction': 0.9500000000000001,
 'lambda_l1': 0.008766822936057211,
 'lambda_l2': 0.3679733442683433,
 'learning_rate': 0.037,
 'max_depth': 13,
 'metric': 'rmse',
 'n_estimators': 10000,
 'num_leaves': 63,
 'objective': 'rmse',
 'random_state': 2019,
 'subsample': 0.31}

What was selected on 100% of the data

```
{'feature_fraction': 0.9500000000000001,
 'lambda_l1': 0.008766822936057211,
 'lambda_l2': 0.3679733442683433,
 'learning_rate': 0.037,
 'max_depth': 13,
 'metric': 'rmse',
 'n_estimators': 10000,
 'num_leaves': 63,
 'objective': 'rmse',
 'random_state': 2019,
 'subsample': 0.31}```

In [134]:
# best_params={'feature_fraction': 0.58,
#  'lambda_l1': 0.45619796864269707,
#  'lambda_l2': 0.033257384218246686,
#  'learning_rate': 0.007,
#  'max_depth': 14,
#  'metric': 'multiclass',
#  'n_estimators': 10000,
#  'num_classes': 4,
#  'num_leaves': 31,
#  'objective': 'multiclass',
#  'random_state': 2019,
#  'subsample': 0.9500000000000001}

In [31]:
# with open("../dsb2019/models/regression_baseline_params.json", "w") as f:
#     json.dump(best_params, f)

In [32]:
baseline_model=train_baseline(train_features.drop(["installation_id", "accuracy_group"], axis=1), train_features.accuracy_group.values, 
               params=best_params)

Training until validation scores don't improve for 2000 rounds
[100]	valid_0's rmse: 0.988984	valid_0's quad_kappa: 0.515772
[200]	valid_0's rmse: 0.99006	valid_0's quad_kappa: 0.531591
[300]	valid_0's rmse: 0.994043	valid_0's quad_kappa: 0.529496
[400]	valid_0's rmse: 0.99734	valid_0's quad_kappa: 0.532411
[500]	valid_0's rmse: 0.999939	valid_0's quad_kappa: 0.533361
[600]	valid_0's rmse: 1.00358	valid_0's quad_kappa: 0.530457
[700]	valid_0's rmse: 1.00656	valid_0's quad_kappa: 0.53098
[800]	valid_0's rmse: 1.00873	valid_0's quad_kappa: 0.533389
[900]	valid_0's rmse: 1.01111	valid_0's quad_kappa: 0.533766
[1000]	valid_0's rmse: 1.01385	valid_0's quad_kappa: 0.533646
[1100]	valid_0's rmse: 1.01638	valid_0's quad_kappa: 0.532839
[1200]	valid_0's rmse: 1.01794	valid_0's quad_kappa: 0.532546
[1300]	valid_0's rmse: 1.02023	valid_0's quad_kappa: 0.532289
[1400]	valid_0's rmse: 1.02196	valid_0's quad_kappa: 0.534031
[1500]	valid_0's rmse: 1.0235	valid_0's quad_kappa: 0.536665
[1600]	valid_0'

In [33]:
predictions = cross_validate(train_features, train_features.accuracy_group, make_features_wrapper, partial(train_baseline, params=best_params), 
                             make_predictions)
np.mean([quad_kappa(true, pred) for pred, true in predictions]), [quad_kappa(true, pred) for pred, true in predictions]

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 2000 rounds
[100]	valid_0's rmse: 1.00316	valid_0's quad_kappa: 0.513435
[200]	valid_0's rmse: 1.00496	valid_0's quad_kappa: 0.528577
[300]	valid_0's rmse: 1.01188	valid_0's quad_kappa: 0.523963
[400]	valid_0's rmse: 1.0174	valid_0's quad_kappa: 0.523392
[500]	valid_0's rmse: 1.02133	valid_0's quad_kappa: 0.531636
[600]	valid_0's rmse: 1.02557	valid_0's quad_kappa: 0.526452
[700]	valid_0's rmse: 1.02913	valid_0's quad_kappa: 0.524162
[800]	valid_0's rmse: 1.03247	valid_0's quad_kappa: 0.520864
[900]	valid_0's rmse: 1.03517	valid_0's quad_kappa: 0.515501
[1000]	valid_0's rmse: 1.03666	valid_0's quad_kappa: 0.513873
[1100]	valid_0's rmse: 1.03767	valid_0's quad_kappa: 0.513208
[1200]	valid_0's rmse: 1.03887	valid_0's quad_kappa: 0.515455
[1300]	valid_0's rmse: 1.0398	valid_0's quad_kappa: 0.520122
[1400]	valid_0's rmse: 1.04103	valid_0's quad_kappa: 0.520937
[1500]	valid_0's rmse: 1.04226	valid_0's quad_kappa: 0.521981
[1600]	valid_0's r

(0.5040995962322988,
 [0.5238925645826822,
  0.5105528877154102,
  0.5070722996279785,
  0.48752747550490694,
  0.4914527537305162])

In [34]:
baseline_model.save_model(str(MODELS_DIR / "regression_baseline.lgb"))

In [45]:
features, target = make_features(train_features)
prediction=baseline_model.predict(features)
clf = ThresholdClassifier()
clf.fit(prediction, target)

100%|██████████| 1000/1000 [00:22<00:00, 44.03it/s, best loss: -0.6998940746763216]


ThresholdClassifier(n_iter=1000, random_state=2019)

In [46]:
clf.coef_

[1.1184111455664985, 1.7332944385971683, 2.2191294740561323]

In [25]:
mean_score, cv_scores = (0.5216346950551788,
 [0.5537180482275439,
  0.5369835700820295,
  0.5238957300540257,
  0.49549097214072135,
  0.49808515477157345])
name = "Game baseline"
notebook_path="notebooks/Game baseline.ipynb"
submission_path="notebooks/submissions/Game baseline submission.ipynb"
track_experiment(name, mean_score, cv_scores, notebook_path)
track_submission_info(name, submission_path, 0.445)

,name,time,mean_score,cv_scores,notebook_path,submission_path,submission_score
0,Game baseline,2019-11-19 16:51:22.976533+04:00,0.521635,"[0.5537180482275439, 0.5369835700820295, 0.523...",notebooks/Game baseline.ipynb,notebooks/submissions/Game baseline submission...,0.445
1,Time + calendar features as is,2019-11-04 02:46:40.201648+04:00,0.509131,"[0.53695404946506, 0.5242692903114117, 0.50829...",Time features.ipynb,NaN,NaN
